In [16]:
import pandas as pd
import numpy as np
import itertools as it

Defining class for Naives bayes for spam=1 and ham=0 according to class context

In [15]:
def group_class(naive_bayes):
    naive_bayes_class_0 = naive_bayes[naive_bayes['CLASS'] == 0]
    naive_bayes_class_0 = naive_bayes_class_0.drop(columns=['CLASS'])
    naive_bayes_class_1 = naive_bayes[naive_bayes['CLASS'] == 1]
    naive_bayes_class_1 = naive_bayes_class_1.drop(columns=['CLASS'])
    return naive_bayes_class_0, naive_bayes_class_1

Conditional probabilty calculation and prior class (spam and ham)

In [17]:
def train_naive_bayes(naive_bayes_train, alpha):
    naive_bayes_class_0, naive_bayes_class_1 = group_class(naive_bayes_train)

    prior_class_0 = (float(len(naive_bayes_class_0.index)) / (len(naive_bayes_class_0.index) + len(naive_bayes_class_1.index)))
    prior_class_1 = (float(len(naive_bayes_class_1.index)) / (len(naive_bayes_class_0.index) + len(naive_bayes_class_1.index)))

    probability_naive_bayes_0 = (naive_bayes_class_0.sum(axis=0, skipna=True) + alpha) / (
                naive_bayes_class_0.values.sum() + len(naive_bayes_class_0.columns))

    probability_naive_bayes_1 = (naive_bayes_class_1.sum(axis=0, skipna=True) + alpha) / (
                naive_bayes_class_1.values.sum() + len(naive_bayes_class_1.columns))

    return prior_class_0, prior_class_1, probability_naive_bayes_0, probability_naive_bayes_1

For the given data predict classes P ωi | x j()∝P x|ωi()P ωi()=P ωi()P xi |ωj()j=1

In [18]:
def predict_naive_bayes(prior_class_0, prior_class_1, probability_naive_bayes_0, probability_naive_bayes_1, naive_bayes_test):
    naive_bayes_test = naive_bayes_test.drop(columns=['CLASS'])

    predictions = []

    for index, row in naive_bayes_test.iterrows():
        row_prediction_0 = prior_class_0
        row_prediction_1 = prior_class_1
        i = 0
        for item in row:
            if item != 0:
                row_prediction_0 = row_prediction_0 * probability_naive_bayes_0.iloc[i]
                row_prediction_1 = row_prediction_1 * probability_naive_bayes_1.iloc[i]
            i = i + 1

        if row_prediction_0 >= row_prediction_1:
            predictions.append(0)
        else:
            predictions.append(1)

    return predictions

To define and get f measure score that is required 

In [19]:
def accuracy_naive_bayes(naive_bayes_predictions, naive_bayes_act_class):
    TP, TN, FP, FN = 0.0, 0.0, 0.0, 0.0

    for p, a in it.zip_longest(naive_bayes_predictions, naive_bayes_act_class):
        if p == 0 and a == 0:
            TP = TP + 1
        elif p == 1 and a == 1:
            TN = TN + 1
        elif p == 0 and a == 1:
            FP = FP + 1
        elif p == 1 and a == 0:
            FN = FN + 1
    pre = TP / (TP + FP)
    rec = TP / (TP + FN)
    f_measure = (2 * pre * rec) / (pre + rec)
    return f_measure


Reading the subjects email file and implementing it

In [20]:
sub = pd.read_csv('dbworld_subjects_stemmed.csv')
sub = sub.drop(columns=['id'])
sub_train = sub.sample(frac=0.8, random_state=70)
sub_test = sub.drop(sub_train.index)
prior_sub_class_0, prior_sub_class_1, probability_sub_0, probability_sub_1 = train_naive_bayes(sub_train, 1)
sub_predictions = predict_naive_bayes(prior_sub_class_0, prior_sub_class_1, probability_sub_0, probability_sub_1, sub_test)
sub_act_class = []

Assiging label for each member, getting the f measure score 

In [12]:
for item in sub_test.iloc[:, -1]:
    sub_act_class.append(item)
sub_f_measure = accuracy_naive_bayes(sub_predictions, sub_act_class)
print ("The f measure score for subjects:",sub_f_measure)

The f measure score for subjects: 1.0


In [13]:
bod = pd.read_csv('dbworld_bodies_stemmed.csv')
bod = bod.drop(columns=['id'])
bod_train = bod.sample(frac=0.8, random_state=70)
bod_test = bod.drop(bod_train.index)
prior_bod_class_0, prior_bod_class_1, probability_bod_0, probability_bod_1 = train_naive_bayes(bod_train, 1)
bod_predictions = predict_naive_bayes(prior_bod_class_0, prior_bod_class_1, probability_bod_0, probability_bod_1, bod_test)
bod_act_class = []

In [14]:
for item in bod_test.iloc[:, -1]:
    bod_act_class.append(item)
bod_f_measure = accuracy_naive_bayes(bod_predictions, bod_act_class)
print ("The f measure score for body:",bod_f_measure)

The f measure score for body: 0.8571428571428571
